In [44]:
from torchvision import datasets
from torchvision.transforms import ToTensor

In [45]:
train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True
)
test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download = True
)

In [46]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [47]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [48]:
train_data.data.shape

torch.Size([60000, 28, 28])

In [49]:
train_data.targets.size()

torch.Size([60000])

In [50]:
from torch.utils.data import DataLoader

loaders = {

    'train': DataLoader(train_data,
                        batch_size=100,
                        shuffle=True,
                        num_workers=1),
    
    'test': DataLoader(test_data,
                        batch_size=100,
                        shuffle=True,
                        num_workers=1)

}

In [51]:
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x34e35f590>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x34e35f920>}

In [67]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        self = self.fc2(x)

        return F.softmax(x)

In [68]:
import torch
print(torch.backends.mps.is_available())

# device = torch.device('mps' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

model = CNN().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(loaders['train']):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(loaders["train"].dataset)} ({100. * batch_idx/len(loaders["train"]):.0f}%)]\t{loss.item():.6f}')

def test():
    model.eval()

    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in loaders['test']:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(loaders['test'].dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy {correct}/{len(loaders["test"].dataset)} ({100. * correct / len(loaders["test"].dataset):.0f}%\n)')
    

True


In [69]:
import time

#device is already set up to CPU

start_time_cpu = time.time()

for epoch in range(1,11):
    train(epoch)
    test()

end_time_cpu = time.time() 
total_time_cpu = end_time_cpu - start_time_cpu
#total_time_cpu


/var/folders/8_/qqhcw4rd4m72212dbpl453gh0000gn/T/ipykernel_47958/845714647.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Train Epoch: 1 [0/60000 (0%)]	3.912134
Train Epoch: 1 [1280/60000 (2%)]	3.808360
Train Epoch: 1 [2560/60000 (4%)]	3.706008
Train Epoch: 1 [3840/60000 (6%)]	3.596577
Train Epoch: 1 [5120/60000 (9%)]	3.580451
Train Epoch: 1 [6400/60000 (11%)]	3.640138
Train Epoch: 1 [7680/60000 (13%)]	3.618248
Train Epoch: 1 [8960/60000 (15%)]	3.616766
Train Epoch: 1 [10240/60000 (17%)]	3.629657
Train Epoch: 1 [11520/60000 (19%)]	3.618950
Train Epoch: 1 [12800/60000 (21%)]	3.480084
Train Epoch: 1 [14080/60000 (23%)]	3.607445
Train Epoch: 1 [15360/60000 (26%)]	3.483847
Train Epoch: 1 [16640/60000 (28%)]	3.504645
Train Epoch: 1 [17920/60000 (30%)]	3.458641
Train Epoch: 1 [19200/60000 (32%)]	3.540498
Train Epoch: 1 [20480/60000 (34%)]	3.590261
Train Epoch: 1 [21760/60000 (36%)]	3.569579
Train Epoch: 1 [23040/60000 (38%)]	3.535046
Train Epoch: 1 [24320/60000 (41%)]	3.551995
Train Epoch: 1 [25600/60000 (43%)]	3.494175
Train Epoch: 1 [26880/60000 (45%)]	3.551357
Train Epoch: 1 [28160/60000 (47%)]	3.501405
Trai

In [70]:
device = torch.device('mps') #switching device to GPU

model = CNN().to(device)

start_time_gpu = time.time()

for epoch in range(1,11):
    train(epoch)
    test()

end_time_gpu = time.time() 
total_time_gpu = end_time_gpu - start_time_gpu
#total_time_gpu


/var/folders/8_/qqhcw4rd4m72212dbpl453gh0000gn/T/ipykernel_47958/845714647.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Train Epoch: 1 [0/60000 (0%)]	3.909415
Train Epoch: 1 [1280/60000 (2%)]	3.913392
Train Epoch: 1 [2560/60000 (4%)]	3.906799
Train Epoch: 1 [3840/60000 (6%)]	3.912159
Train Epoch: 1 [5120/60000 (9%)]	3.910404
Train Epoch: 1 [6400/60000 (11%)]	3.909650
Train Epoch: 1 [7680/60000 (13%)]	3.909739
Train Epoch: 1 [8960/60000 (15%)]	3.911198
Train Epoch: 1 [10240/60000 (17%)]	3.910629
Train Epoch: 1 [11520/60000 (19%)]	3.914146
Train Epoch: 1 [12800/60000 (21%)]	3.911399
Train Epoch: 1 [14080/60000 (23%)]	3.911247
Train Epoch: 1 [15360/60000 (26%)]	3.908560
Train Epoch: 1 [16640/60000 (28%)]	3.908349
Train Epoch: 1 [17920/60000 (30%)]	3.912389
Train Epoch: 1 [19200/60000 (32%)]	3.913354
Train Epoch: 1 [20480/60000 (34%)]	3.910456
Train Epoch: 1 [21760/60000 (36%)]	3.912143
Train Epoch: 1 [23040/60000 (38%)]	3.911807
Train Epoch: 1 [24320/60000 (41%)]	3.910114
Train Epoch: 1 [25600/60000 (43%)]	3.911507
Train Epoch: 1 [26880/60000 (45%)]	3.911161
Train Epoch: 1 [28160/60000 (47%)]	3.912561
Trai

In [97]:

print(f"Time training for CPU is: {total_time_cpu:.2f} seconds.")
print(f"Time training for GPU is: {total_time_gpu:.2f} seconds.\n")

if total_time_cpu > total_time_gpu:
    print(f"The GPU outpreformed the CPU for: {total_time_cpu-total_time_gpu:.2f} seconds.")
    print(f"The GPU was: {total_time_cpu/total_time_gpu:2f} times faster.")
    print(f"The GPU was: {(total_time_cpu - total_time_gpu) / total_time_cpu * 100:2f} % faster.")
else:
    print(f"The CPU outpreformed the CPU for: {total_time_gpu-total_time_cpu:.2f} seconds. or {total_time_cpu/total_time_gpu*100:2f}%.")
    print(f"The CPU was: {total_time_gpu/total_time_cpu:2f} times faster.")
    print(f"The CPU was: {(total_time_gpu - total_time_cpu) / total_time_gpu * 100:2f} % faster.")


Time training for CPU is: 67.75 seconds.
Time training for GPU is: 48.18 seconds.

The GPU outpreformed the CPU for: 19.57 seconds.
The GPU was: 1.406161 times faster.
The GPU was: 28.884371 % faster.
